# Recommender Systems 2021/20

### Practice - SLIM MSE implemented with Python and Cython


### Cython is a superset of Python, allowing you to use C-like operations and import C code. Cython files (.pyx) are compiled and support static typing.

### Why do we use it (or any other compiled language)? If the code is written properly it is fast... I mean, FAST

In [1]:
import time
import numpy as np

### Let's implement something simple

In [2]:
def isPrime(n):
    
    i = 2
    
    # Usually you loop up to sqrt(n)
    while i < n:
        if n % i == 0:
            return False
        
        i += 1
        
    return True

In [3]:
print("Is prime 2? {}".format(isPrime(2)))
print("Is prime 3? {}".format(isPrime(3)))
print("Is prime 5? {}".format(isPrime(5)))
print("Is prime 15? {}".format(isPrime(15)))
print("Is prime 20? {}".format(isPrime(20)))

Is prime 2? True
Is prime 3? True
Is prime 5? True
Is prime 15? False
Is prime 20? False


In [4]:
start_time = time.time()

result = isPrime(80000023)

print("Is Prime 80000023? {}, time required {:.2f} sec".format(result, time.time()-start_time))

Is Prime 80000023? True, time required 9.58 sec


#### Load Cython magic command, this takes care of the compilation step. If you are writing code outside Jupyter you'll have to compile using other tools. See at the end of the notebook for details.

In [5]:
%load_ext Cython

#### Declare Cython function, paste the same code as before. The function will be compiled and then executed with a Python interface

In [6]:
%%cython
def isPrime(n):
    
    i = 2
    
    # Usually you loop up to sqrt(n)
    while i < n:
        if n % i == 0:
            return False
        
        i += 1
        
    return True

In [7]:
start_time = time.time()

result = isPrime(80000023)

print("Is Prime 80000023? {}, time required {:.2f} sec".format(result, time.time()-start_time))

Is Prime 80000023? True, time required 4.48 sec


#### As you can see by just compiling the same code we got some improvement.
#### To go seriously higher, we have to use some static tiping

In [8]:
%%cython
# Declare the tipe of the arguments
def isPrime(long n):
    
    # Declare index of for loop
    cdef long i
    
    i = 2
    
    # Usually you loop up to sqrt(n)
    while i < n:
        if n % i == 0:
            return False
        
        i += 1
        
    return True

In [9]:
start_time = time.time()

result = isPrime(80000023)

print("Is Prime 80000023? {}, time required {:.2f} sec".format(result, time.time()-start_time))

Is Prime 80000023? True, time required 0.22 sec


#### Cython code with two tipe declaration, for n and i, runs 50x faster than Python

#### Main benefits of Cython:
* Compiled, no interpreter
* Static typing, no overhead
* Fast loops, no need to vectorize. Vectorization sometimes performes lots of useless operations
* Numpy, which is fast in python, when opertions are not vectorizable often becomes slooooow compared to a carefully written Cython code

## SLIM MSE with Cython

#### Load the usual data.

In [10]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

data_reader = Movielens10MReader()
data_loaded = data_reader.load_data()

URM_all = data_loaded.get_URM_all()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

Movielens10M: Verifying data consistency...
Movielens10M: Verifying data consistency... Passed!
DataReader: current dataset is: <class 'Data_manager.Dataset.Dataset'>
	Number of items: 10681
	Number of users: 69878
	Number of interactions in URM_all: 10000054
	Value range in URM_all: 0.50-5.00
	Interaction density: 1.34E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.43E+02
		 Max: 7.36E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 9.36E+02
		 Max: 3.49E+04
	Gini Index: 0.57

	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 20, feature occurrences: 21564, density 1.01E-01
	ICM name: ICM_tags, Value range: 1.00 / 69.00, Num features: 10217, feature occurrences: 108563, density 9.95E-04
	ICM name: ICM_all, Value range: 1.00 / 69.00, Num features: 10237, feature occurrences: 130127, density 1.19E-03




In [11]:
URM_train

<69878x10681 sparse matrix of type '<class 'numpy.float64'>'
	with 8000975 stored elements in Compressed Sparse Row format>

### What do we need for a SLIM MSE?

* Item-Item similarity matrix
* Computing prediction
* Update rule
* Training loop and some patience


In [12]:
n_users, n_items = URM_train.shape

## Step 1: We create a dense similarity matrix, initialized as zero

In [13]:
item_item_S = np.zeros((n_items, n_items), dtype = np.float)
item_item_S

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Step 2: We sample an interaction and compute the prediction of the current SLIM model

In [14]:
URM_train_coo = URM_train.tocoo()

sample_index = np.random.randint(URM_train_coo.nnz)
sample_index

3787221

In [15]:
user_id = URM_train_coo.row[sample_index]
item_id = URM_train_coo.col[sample_index]
true_rating = URM_train_coo.data[sample_index]

(user_id, item_id, true_rating)

(33017, 279, 5.0)

In [16]:
predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]
predicted_rating

0.0

#### The first predicted rating is zero, of course, the model is "empty"

### Step 3: We compute the prediction error and update the item-item similarity

In [17]:
prediction_error = true_rating - predicted_rating
prediction_error

5.0

### The error is positive, so we need to increase the prediction our model computes. Meaning, we have to increase the values in the item-item similarity matrix

### Which item similarities we modify? Only those we used to compute the prediction, i.e., only the items in the profile of the sampled user. 

In [18]:
items_in_user_profile = URM_train[user_id].indices
items_in_user_profile

array([   2,    3,    4,    5,    7,    9,   11,   14,   16,   17,   18,
         19,   20,   22,   24,   26,   27,   31,   32,   33,   34,   35,
         36,   37,   38,   40,   43,   48,   51,   52,   55,   56,   60,
         61,   62,   63,   71,   74,   77,   78,   79,   81,   82,   84,
         85,   91,   93,   94,   95,   96,   97,  101,  102,  130,  133,
        139,  146,  147,  148,  156,  163,  165,  171,  175,  176,  177,
        178,  179,  183,  185,  186,  187,  188,  190,  192,  195,  197,
        198,  199,  201,  202,  204,  206,  209,  212,  213,  234,  235,
        238,  241,  242,  244,  245,  246,  248,  252,  258,  259,  261,
        271,  277,  279,  284,  285,  292,  294,  296,  301,  302,  309,
        312,  315,  321,  323,  324,  329,  331,  336,  338,  340,  345,
        346,  358,  369,  373,  377,  380,  385,  389,  390,  391,  401,
        402,  403,  406,  411,  413,  415,  416,  417,  418,  420,  422,
        423,  425,  426,  428,  436,  437,  440,  4

In [19]:
ratings_in_user_profile = URM_train[user_id].data
ratings_in_user_profile

array([3. , 3.5, 3. , 3. , 5. , 5. , 3.5, 5. , 4. , 4. , 4. , 5. , 4. ,
       5. , 5. , 5. , 4. , 3. , 4. , 3.5, 5. , 4. , 4. , 5. , 4. , 4. ,
       4.5, 3.5, 4. , 4. , 3. , 4. , 4.5, 3.5, 5. , 4.5, 4. , 4. , 4. ,
       4. , 4. , 3. , 4. , 3. , 4.5, 4. , 5. , 4. , 4. , 5. , 4. , 4. ,
       4.5, 5. , 5. , 5. , 5. , 5. , 5. , 5. , 4.5, 5. , 5. , 5. , 4. ,
       5. , 5. , 5. , 5. , 4. , 3. , 3.5, 4. , 4. , 5. , 5. , 3.5, 5. ,
       4. , 4. , 3.5, 3. , 5. , 3.5, 3. , 5. , 5. , 5. , 4. , 4.5, 4.5,
       5. , 4. , 4. , 3.5, 3.5, 4. , 5. , 5. , 4. , 5. , 5. , 4. , 3. ,
       4. , 3.5, 4. , 3. , 5. , 3.5, 4. , 4. , 3. , 3. , 4. , 4. , 3. ,
       4. , 4. , 3. , 3. , 3. , 4. , 3. , 3. , 2. , 4. , 3. , 4. , 4. ,
       3.5, 5. , 3. , 3. , 4. , 4. , 4. , 5. , 4. , 4.5, 3. , 3.5, 4. ,
       3.5, 4. , 5. , 4. , 1. , 5. , 1. , 2.5, 4. , 4.5, 3. , 3. , 4. ,
       4. , 4.5, 4. , 4.5, 3. , 3. , 3.5, 3. , 3. , 4. , 5. , 4. , 4. ,
       4. , 4. , 4. , 4. , 4. , 3. , 4.5, 4. , 4. , 4. , 3. , 4.

#### Apply the update rule

In [20]:
item_item_S[items_in_user_profile,item_id]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [21]:
learning_rate = 1e-4

item_item_S[items_in_user_profile,item_id] += learning_rate * prediction_error * ratings_in_user_profile

In [22]:
item_item_S[items_in_user_profile,item_id]

array([0.0015 , 0.00175, 0.0015 , 0.0015 , 0.0025 , 0.0025 , 0.00175,
       0.0025 , 0.002  , 0.002  , 0.002  , 0.0025 , 0.002  , 0.0025 ,
       0.0025 , 0.0025 , 0.002  , 0.0015 , 0.002  , 0.00175, 0.0025 ,
       0.002  , 0.002  , 0.0025 , 0.002  , 0.002  , 0.00225, 0.00175,
       0.002  , 0.002  , 0.0015 , 0.002  , 0.00225, 0.00175, 0.0025 ,
       0.00225, 0.002  , 0.002  , 0.002  , 0.002  , 0.002  , 0.0015 ,
       0.002  , 0.0015 , 0.00225, 0.002  , 0.0025 , 0.002  , 0.002  ,
       0.0025 , 0.002  , 0.002  , 0.00225, 0.0025 , 0.0025 , 0.0025 ,
       0.0025 , 0.0025 , 0.0025 , 0.0025 , 0.00225, 0.0025 , 0.0025 ,
       0.0025 , 0.002  , 0.0025 , 0.0025 , 0.0025 , 0.0025 , 0.002  ,
       0.0015 , 0.00175, 0.002  , 0.002  , 0.0025 , 0.0025 , 0.00175,
       0.0025 , 0.002  , 0.002  , 0.00175, 0.0015 , 0.0025 , 0.00175,
       0.0015 , 0.0025 , 0.0025 , 0.0025 , 0.002  , 0.00225, 0.00225,
       0.0025 , 0.002  , 0.002  , 0.00175, 0.00175, 0.002  , 0.0025 ,
       0.0025 , 0.00

### Let's check what the new prediction for the same user-item interaction would be

In [23]:
predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]
predicted_rating

3.6118750000000057

### The value is not zero anymore, but higher, we are moving in the right direction

### And now? Sample another interaction and repeat... a lot of times

### Let's put all together in a training loop.

In [24]:
URM_train_coo = URM_train.tocoo()
item_item_S = np.zeros((n_items, n_items), dtype = np.float)

learning_rate = 1e-6

loss = 0.0
start_time = time.time()
for sample_num in range(100000):
    
    # Randomly pick sample
    sample_index = np.random.randint(URM_train_coo.nnz)

    user_id = URM_train_coo.row[sample_index]
    item_id = URM_train_coo.col[sample_index]
    true_rating = URM_train_coo.data[sample_index]

    # Compute prediction
    predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]
        
    # Compute prediction error, or gradient
    prediction_error = true_rating - predicted_rating
    loss += prediction_error**2
    
    # Update model, in this case the similarity
    items_in_user_profile = URM_train[user_id].indices
    ratings_in_user_profile = URM_train[user_id].data
    item_item_S[items_in_user_profile,item_id] += learning_rate * prediction_error * ratings_in_user_profile
    
    # Print some stats
    if (sample_num +1)% 5000 == 0:
        elapsed_time = time.time() - start_time
        samples_per_second = (sample_num +1)/elapsed_time
        print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/(sample_num +1), samples_per_second))


Iteration 5000 in 2.52 seconds, loss is 13.49. Samples per second 1981.70
Iteration 10000 in 4.51 seconds, loss is 13.41. Samples per second 2216.32
Iteration 15000 in 6.53 seconds, loss is 13.37. Samples per second 2295.72
Iteration 20000 in 8.51 seconds, loss is 13.35. Samples per second 2349.94
Iteration 25000 in 10.50 seconds, loss is 13.31. Samples per second 2381.21
Iteration 30000 in 12.49 seconds, loss is 13.29. Samples per second 2402.15
Iteration 35000 in 14.49 seconds, loss is 13.27. Samples per second 2415.33
Iteration 40000 in 16.42 seconds, loss is 13.24. Samples per second 2435.49
Iteration 45000 in 18.55 seconds, loss is 13.20. Samples per second 2426.42
Iteration 50000 in 20.67 seconds, loss is 13.17. Samples per second 2419.29
Iteration 55000 in 22.80 seconds, loss is 13.16. Samples per second 2412.33
Iteration 60000 in 24.88 seconds, loss is 13.14. Samples per second 2411.42
Iteration 65000 in 26.83 seconds, loss is 13.12. Samples per second 2422.83
Iteration 70000 i

### What do we see? The loss oscillates over time, sometimes it goes down sometimes up.
### How long do we train such a model?

* An epoch: a complete loop over all the train data
* Usually you train for multiple epochs. Depending on the algorithm and data 10s or 100s of epochs.

In [25]:
def train_one_epoch(URM_train, item_item_S, learning_rate):
    
    URM_train_coo = URM_train.tocoo()

    loss = 0.0
    start_time = time.time()
    for sample_num in range(URM_train.nnz):

        # Randomly pick sample
        sample_index = np.random.randint(URM_train_coo.nnz)

        user_id = URM_train_coo.row[sample_index]
        item_id = URM_train_coo.col[sample_index]
        true_rating = URM_train_coo.data[sample_index]

        # Compute prediction
        predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]

        # Compute prediction error, or gradient
        prediction_error = true_rating - predicted_rating
        loss += prediction_error**2

        # Update model, in this case the similarity
        items_in_user_profile = URM_train[user_id].indices
        ratings_in_user_profile = URM_train[user_id].data
        item_item_S[items_in_user_profile,item_id] += learning_rate * prediction_error * ratings_in_user_profile

        # Print some stats
        if (sample_num +1)% 5000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = (sample_num +1)/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/(sample_num +1), samples_per_second))
         
            # Stop training because this implementation is too slow
            print("\tStopping the epoch early because this implementation is too slow")
            return item_item_S
            
    return item_item_S

In [26]:
n_items = URM_train.shape[1]
learning_rate = 1e-6
    
item_item_S = np.zeros((n_items, n_items), dtype = np.float)

for n_epoch in range(5):
    item_item_S = train_one_epoch(URM_train, item_item_S, learning_rate)
    

Iteration 5000 in 2.20 seconds, loss is 13.45. Samples per second 2271.72
	Stopping the epoch early because this implementation is too slow
Iteration 5000 in 1.94 seconds, loss is 13.44. Samples per second 2583.97
	Stopping the epoch early because this implementation is too slow
Iteration 5000 in 1.93 seconds, loss is 13.30. Samples per second 2586.25
	Stopping the epoch early because this implementation is too slow
Iteration 5000 in 1.91 seconds, loss is 13.24. Samples per second 2620.69
	Stopping the epoch early because this implementation is too slow
Iteration 5000 in 1.94 seconds, loss is 13.35. Samples per second 2582.58
	Stopping the epoch early because this implementation is too slow


In [27]:
item_item_S

array([[2.99929130e-05, 5.32433053e-05, 4.63581199e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.09904936e-05, 5.99211075e-04, 2.12381781e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.99923892e-05, 2.49503284e-04, 5.10311713e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

### How do we use this similarity? As in a simple item-based KNN

### Let's estimate the training time. Say we train for 10 epochs and we have 8M interactions in the train data...

In [28]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 34120.00 seconds, or 568.67 minutes


### ... ehm, 10 hours
### Unacceptable!

### Let's rewrite the loop with some smarter use of the data structures. In particular:
* Use the indptr/indices data structures to get the seen items
* Not much else we can do with this tools

In [29]:
URM_train_coo = URM_train.tocoo()
item_item_S = np.zeros((n_items, n_items), dtype = np.float)

learning_rate = 1e-6

loss = 0.0
start_time = time.time()
for sample_num in range(100000):
    
    # Randomly pick sample
    sample_index = np.random.randint(URM_train_coo.nnz)

    user_id = URM_train_coo.row[sample_index]
    item_id = URM_train_coo.col[sample_index]
    true_rating = URM_train_coo.data[sample_index]

    # Compute prediction
    predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]
        
    # Compute prediction error, or gradient
    prediction_error = true_rating - predicted_rating
    loss += prediction_error**2
    
    # Update model, in this case the similarity
    items_in_user_profile = URM_train.indices[URM_train.indptr[user_id]:URM_train.indptr[user_id+1]]
    ratings_in_user_profile = URM_train.data[URM_train.indptr[user_id]:URM_train.indptr[user_id+1]]
    item_item_S[items_in_user_profile,item_id] += learning_rate * prediction_error * ratings_in_user_profile
    
    # Print some stats
    if (sample_num +1)% 5000 == 0:
        elapsed_time = time.time() - start_time
        samples_per_second = (sample_num +1)/elapsed_time
        print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/(sample_num +1), samples_per_second))


Iteration 5000 in 1.64 seconds, loss is 13.36. Samples per second 3046.01
Iteration 10000 in 3.09 seconds, loss is 13.36. Samples per second 3236.43
Iteration 15000 in 4.48 seconds, loss is 13.35. Samples per second 3349.89
Iteration 20000 in 5.88 seconds, loss is 13.34. Samples per second 3401.82
Iteration 25000 in 7.30 seconds, loss is 13.31. Samples per second 3423.59
Iteration 30000 in 8.74 seconds, loss is 13.27. Samples per second 3432.38
Iteration 35000 in 10.16 seconds, loss is 13.22. Samples per second 3444.04
Iteration 40000 in 11.60 seconds, loss is 13.19. Samples per second 3448.66
Iteration 45000 in 13.00 seconds, loss is 13.17. Samples per second 3460.66
Iteration 50000 in 14.43 seconds, loss is 13.15. Samples per second 3463.92
Iteration 55000 in 15.83 seconds, loss is 13.12. Samples per second 3474.56
Iteration 60000 in 17.26 seconds, loss is 13.10. Samples per second 3475.42
Iteration 65000 in 18.68 seconds, loss is 13.07. Samples per second 3480.08
Iteration 70000 in 

In [30]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 22975.86 seconds, or 382.93 minutes


### We now got 7 hours, just as bad as before

### Let's see what we can do with Cython
### First step, just compile it. We do not have the data at compile time, so we put the loop in a function

In [31]:
%%cython
import numpy as np
import time

def do_some_training(URM_train):
    
    URM_train_coo = URM_train.tocoo()
    n_items = URM_train.shape[1]
    
    item_item_S = np.zeros((n_items, n_items), dtype = np.float16)

    learning_rate = 1e-6

    loss = 0.0
    start_time = time.time()
    for sample_num in range(100000):

        # Randomly pick sample
        sample_index = np.random.randint(URM_train_coo.nnz)

        user_id = URM_train_coo.row[sample_index]
        item_id = URM_train_coo.col[sample_index]
        true_rating = URM_train_coo.data[sample_index]

        # Compute prediction
        predicted_rating = URM_train[user_id].dot(item_item_S[:,item_id])[0]

        # Compute prediction error, or gradient
        prediction_error = true_rating - predicted_rating
        loss += prediction_error**2

        # Update model, in this case the similarity
        items_in_user_profile = URM_train.indices[URM_train.indptr[user_id]:URM_train.indptr[user_id+1]]
        ratings_in_user_profile = URM_train.data[URM_train.indptr[user_id]:URM_train.indptr[user_id+1]]
        item_item_S[items_in_user_profile,item_id] += learning_rate * prediction_error * ratings_in_user_profile

        # Print some stats
        if (sample_num +1)% 5000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = (sample_num +1)/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/(sample_num +1), samples_per_second))
    
    return loss, samples_per_second

In [32]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 5000 in 1.43 seconds, loss is 13.38. Samples per second 3505.21
Iteration 10000 in 2.86 seconds, loss is 13.34. Samples per second 3501.29
Iteration 15000 in 4.30 seconds, loss is 13.30. Samples per second 3488.81
Iteration 20000 in 5.71 seconds, loss is 13.30. Samples per second 3500.03
Iteration 25000 in 7.20 seconds, loss is 13.30. Samples per second 3473.65
Iteration 30000 in 8.61 seconds, loss is 13.28. Samples per second 3484.30
Iteration 35000 in 10.06 seconds, loss is 13.24. Samples per second 3478.57
Iteration 40000 in 11.56 seconds, loss is 13.22. Samples per second 3459.54
Iteration 45000 in 13.01 seconds, loss is 13.20. Samples per second 3457.90
Iteration 50000 in 14.45 seconds, loss is 13.18. Samples per second 3459.66
Iteration 55000 in 15.91 seconds, loss is 13.14. Samples per second 3457.15
Iteration 60000 in 17.38 seconds, loss is 13.10. Samples per second 3453.15
Iteration 65000 in 18.88 seconds, loss is 13.08. Samples per second 3442.80
Iteration 70000 in 

In [33]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 23524.87 seconds, or 392.08 minutes


### Still far too time consuming
### The compiler is just porting in C all operations that the python interpreter would have to perform, dynamic tiping included. Have a look at the html reports in the Cython_examples folder

### Now try to add some types: If you use a variable only as a C object, use primitive tipes

* cdef int namevar
* cdef double namevar
* cdef float namevar
* cdef double[:] singledimensionarray
* cdef double[:,:] bidimensionalmatrix


### We now use types for all main variables


In [34]:
%%cython
import numpy as np
import time

def do_some_training(URM_train):

    URM_train_coo = URM_train.tocoo()
    n_items = URM_train.shape[1]

    cdef double[:,:] item_item_S = np.zeros((n_items, n_items), dtype = np.float)
    cdef double learning_rate = 1e-6
    cdef double loss = 0.0
    cdef long start_time = time.time()
    cdef double true_rating, predicted_rating, prediction_error, profile_rating
    cdef int[:] items_in_user_profile
    cdef double[:] ratings_in_user_profile
    cdef int index, sample_num, user_id, item_id, profile_item_id

    for sample_num in range(100000):

        # Randomly pick sample
        sample_index = np.random.randint(URM_train_coo.nnz)

        user_id = URM_train_coo.row[sample_index]
        item_id = URM_train_coo.col[sample_index]
        true_rating = URM_train_coo.data[sample_index]

        # Compute prediction
        items_in_user_profile = URM_train.indices[URM_train.indptr[user_id]:URM_train.indptr[user_id+1]]
        ratings_in_user_profile = URM_train.data[URM_train.indptr[user_id]:URM_train.indptr[user_id+1]]
        predicted_rating = 0.0

        for index in range(len(items_in_user_profile)):
            profile_item_id = items_in_user_profile[index]
            profile_rating = ratings_in_user_profile[index]
            predicted_rating += profile_rating * item_item_S[profile_item_id,item_id]

        # Compute prediction error, or gradient
        prediction_error = true_rating - predicted_rating
        loss += prediction_error**2

        # Update model, in this case the similarity
        for index in range(len(items_in_user_profile)):
            profile_item_id = items_in_user_profile[index]
            profile_rating = ratings_in_user_profile[index]
            item_item_S[profile_item_id,item_id] += learning_rate * prediction_error * profile_rating

        # Print some stats
        if (sample_num +1)% 5000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = (sample_num +1)/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/(sample_num +1), samples_per_second))

    return loss, samples_per_second

In [35]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 5000 in 0.58 seconds, loss is 13.30. Samples per second 8652.57
Iteration 10000 in 0.70 seconds, loss is 13.31. Samples per second 14301.68
Iteration 15000 in 0.80 seconds, loss is 13.30. Samples per second 18743.53
Iteration 20000 in 0.90 seconds, loss is 13.33. Samples per second 22275.84
Iteration 25000 in 0.98 seconds, loss is 13.27. Samples per second 25484.44
Iteration 30000 in 1.08 seconds, loss is 13.26. Samples per second 27745.73
Iteration 35000 in 1.17 seconds, loss is 13.25. Samples per second 30019.75
Iteration 40000 in 1.27 seconds, loss is 13.22. Samples per second 31590.23
Iteration 45000 in 1.35 seconds, loss is 13.19. Samples per second 33311.26
Iteration 50000 in 1.44 seconds, loss is 13.18. Samples per second 34829.57
Iteration 55000 in 1.54 seconds, loss is 13.14. Samples per second 35810.92
Iteration 60000 in 1.62 seconds, loss is 13.12. Samples per second 37025.84
Iteration 65000 in 1.72 seconds, loss is 13.08. Samples per second 37773.94
Iteration 7000

In [36]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 1857.88 seconds, or 30.96 minutes


### This is why we use it for machine learning algorithms...

### Some operations are still done with sparse matrices, those cannot be correctly optimized because the compiler does not know how what is the type of the data.

### To address this, we create typed arrays in which we put the URM_train data
For example, this operation: user_id = URM_train_coo.row[sample_index]

Becomes:

cdef int user_id

cdef int[:] URM_train_coo_row = URM_train_coo.row

user_id = URM_train_coo_row[sample_index]


### We can also skip the creation of the items_in_user_profile array and replace the np.random call with the faster native C function rand()

## To obtain a more reliable speed estimate we increase the number of samples and the print step by a factor of 10

In [37]:
%%cython

import numpy as np
import time

from libc.stdlib cimport rand, srand, RAND_MAX

def do_some_training(URM_train):

    URM_train_coo = URM_train.tocoo()
    cdef int n_items = URM_train.shape[1]
    cdef int n_interactions = URM_train.nnz
    cdef int[:] URM_train_coo_row = URM_train_coo.row
    cdef int[:] URM_train_coo_col = URM_train_coo.col
    cdef double[:] URM_train_coo_data = URM_train_coo.data
    cdef int[:] URM_train_indices = URM_train.indices
    cdef int[:] URM_train_indptr = URM_train.indptr
    cdef double[:] URM_train_data = URM_train.data

    cdef double[:,:] item_item_S = np.zeros((n_items, n_items), dtype = np.float)
    cdef double learning_rate = 1e-6
    cdef double loss = 0.0
    cdef long start_time = time.time()
    cdef double true_rating, predicted_rating, prediction_error, profile_rating
    cdef int start_profile, end_profile
    cdef int index, sample_num, user_id, item_id, profile_item_id

    for sample_num in range(1000000):

        # Randomly pick sample
        index = rand() % n_interactions

        user_id = URM_train_coo_row[index]
        item_id = URM_train_coo_col[index]
        true_rating = URM_train_coo_data[index]

        # Compute prediction
        start_profile = URM_train_indptr[user_id]
        end_profile = URM_train_indptr[user_id+1]
        predicted_rating = 0.0

        for index in range(start_profile, end_profile):
            profile_item_id = URM_train_indices[index]
            profile_rating = URM_train_data[index]
            predicted_rating += item_item_S[profile_item_id,item_id] * profile_rating

        # Compute prediction error, or gradient
        prediction_error = true_rating - predicted_rating
        loss += prediction_error**2

        # Update model, in this case the similarity
        for index in range(start_profile, end_profile):
            profile_item_id = URM_train_indices[index]
            profile_rating = URM_train_data[index]
            item_item_S[profile_item_id,item_id] += learning_rate * prediction_error * profile_rating

        # Print some stats
        if (sample_num +1)% 50000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = (sample_num +1)/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/(sample_num +1), samples_per_second))

    return loss, samples_per_second

In [38]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 50000 in 1.08 seconds, loss is 13.55. Samples per second 46332.34
Iteration 100000 in 1.53 seconds, loss is 13.20. Samples per second 65170.61
Iteration 150000 in 2.02 seconds, loss is 12.88. Samples per second 74338.63
Iteration 200000 in 2.50 seconds, loss is 12.56. Samples per second 79870.62
Iteration 250000 in 2.96 seconds, loss is 12.27. Samples per second 84338.99
Iteration 300000 in 3.43 seconds, loss is 11.99. Samples per second 87469.95
Iteration 350000 in 3.92 seconds, loss is 11.72. Samples per second 89373.33
Iteration 400000 in 4.37 seconds, loss is 11.47. Samples per second 91484.56
Iteration 450000 in 4.82 seconds, loss is 11.23. Samples per second 93398.33
Iteration 500000 in 5.28 seconds, loss is 11.01. Samples per second 94777.66
Iteration 550000 in 5.72 seconds, loss is 10.79. Samples per second 96108.86
Iteration 600000 in 6.18 seconds, loss is 10.58. Samples per second 97114.74
Iteration 650000 in 6.62 seconds, loss is 10.39. Samples per second 98219.46


In [39]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 790.91 seconds, or 13.18 minutes


### As the source code gets less readable due to the addition of types and native C functions, it also gets remarkably faster

### We started from a naive python implementation which took 10 hours (2k samples per second) and we now have a Cython one with takes 12 minutes (100k samples per second).

In [40]:
%%cython

import numpy as np
import time

from libc.stdlib cimport rand, srand, RAND_MAX

def train_multiple_epochs(URM_train, learning_rate_input, n_epochs):

    URM_train_coo = URM_train.tocoo()
    cdef int n_items = URM_train.shape[1]
    cdef int n_interactions = URM_train.nnz
    cdef int[:] URM_train_coo_row = URM_train_coo.row
    cdef int[:] URM_train_coo_col = URM_train_coo.col
    cdef double[:] URM_train_coo_data = URM_train_coo.data
    cdef int[:] URM_train_indices = URM_train.indices
    cdef int[:] URM_train_indptr = URM_train.indptr
    cdef double[:] URM_train_data = URM_train.data

    cdef double[:,:] item_item_S = np.zeros((n_items, n_items), dtype = np.float)
    cdef double learning_rate = learning_rate_input
    cdef double loss = 0.0
    cdef long start_time
    cdef double true_rating, predicted_rating, prediction_error, profile_rating
    cdef int start_profile, end_profile
    cdef int index, sample_num, user_id, item_id, profile_item_id
    
    for n_epoch in range(n_epochs):
        
        loss = 0.0
        start_time = time.time()
        
        for sample_num in range(n_interactions):

            # Randomly pick sample
            index = rand() % n_interactions

            user_id = URM_train_coo_row[index]
            item_id = URM_train_coo_col[index]
            true_rating = URM_train_coo_data[index]

            # Compute prediction
            start_profile = URM_train_indptr[user_id]
            end_profile = URM_train_indptr[user_id+1]
            predicted_rating = 0.0

            for index in range(start_profile, end_profile):
                profile_item_id = URM_train_indices[index]
                profile_rating = URM_train_data[index]
                predicted_rating += item_item_S[profile_item_id,item_id] * profile_rating

            # Compute prediction error, or gradient
            prediction_error = true_rating - predicted_rating
            loss += prediction_error**2

            # Update model, in this case the similarity
            for index in range(start_profile, end_profile):
                profile_item_id = URM_train_indices[index]
                profile_rating = URM_train_data[index]
                item_item_S[profile_item_id,item_id] += learning_rate * prediction_error * profile_rating

#             # Print some stats
#             if (sample_num +1)% 1000000 == 0:
#                 elapsed_time = time.time() - start_time
#                 samples_per_second = (sample_num+1)/elapsed_time
#                 print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/(sample_num+1), samples_per_second))

            
        elapsed_time = time.time() - start_time
        samples_per_second = (sample_num+1)/elapsed_time
     
        print("Epoch {} complete in in {:.2f} seconds, loss is {:.3E}. Samples per second {:.2f}".format(n_epoch+1, time.time() - start_time, loss/(sample_num+1), samples_per_second))

    return np.array(item_item_S), loss/(sample_num+1), samples_per_second

In [41]:
n_items = URM_train.shape[1]
learning_rate = 1e-4
    
item_item_S, loss, samples_per_second = train_multiple_epochs(URM_train, learning_rate, 10)

Epoch 1 complete in in 74.06 seconds, loss is 9.571E-02. Samples per second 108034.40
Epoch 2 complete in in 72.56 seconds, loss is 6.914E-05. Samples per second 110261.76
Epoch 3 complete in in 73.60 seconds, loss is 8.308E-06. Samples per second 108705.13
Epoch 4 complete in in 72.17 seconds, loss is 1.526E-06. Samples per second 110856.24
Epoch 5 complete in in 73.03 seconds, loss is 3.504E-07. Samples per second 109560.72
Epoch 6 complete in in 72.54 seconds, loss is 8.842E-08. Samples per second 110298.41
Epoch 7 complete in in 74.30 seconds, loss is 2.332E-08. Samples per second 107681.97
Epoch 8 complete in in 72.96 seconds, loss is 6.066E-09. Samples per second 109665.92
Epoch 9 complete in in 75.92 seconds, loss is 1.717E-09. Samples per second 105388.01
Epoch 10 complete in in 74.96 seconds, loss is 4.639E-10. Samples per second 106735.14


In [42]:
item_item_S

array([[0.07723263, 0.04613929, 0.06089555, ..., 0.        , 0.        ,
        0.        ],
       [0.05279096, 0.24018039, 0.00607983, ..., 0.        , 0.        ,
        0.        ],
       [0.05279096, 0.07732453, 0.31195645, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### Note how the loss decreased?

## How to use Cython outside a notebook

### Step1: Create a .pyx file and write your code

### Step2: Create a compilation script "compileCython.py" with the following content

In [ ]:
# This code will not run in a notebook cell

try:
    from setuptools import setup
    from setuptools import Extension
except ImportError:
    from distutils.core import setup
    from distutils.extension import Extension


from Cython.Distutils import build_ext
import numpy
import sys
import re


if len(sys.argv) != 4:
    raise ValueError("Wrong number of paramethers received. Expected 4, got {}".format(sys.argv))


# Get the name of the file to compile
fileToCompile = sys.argv[1]

# Remove the argument from sys argv in order for it to contain only what setup needs
del sys.argv[1]

extensionName = re.sub("\.pyx", "", fileToCompile)


ext_modules = Extension(extensionName,
                [fileToCompile],
                extra_compile_args=['-O3'],
                include_dirs=[numpy.get_include(),],
                )

setup(
    cmdclass={'build_ext': build_ext},
    ext_modules=[ext_modules]
)


### Step3: Compile your code with the following command 

python compileCython.py Cython_examples\SLIM_MSE_fastest.pyx build_ext --inplace

### Step4: Generate cython report and look for "yellow lines". The report is an .html file which represents how many operations are necessary to translate each python operation in cython code. If a line is white, it has a direct C translation. If it is yellow it will require many indirect steps that will slow down execution. Some of those steps may be inevitable, some may be removed via static typing.

### IMPORTANT: white does not mean fast!! If a system call is involved that part might be slow anyway.

cython -a Cython_examples\SLIM_MSE_fastest.pyx

### Step5: Add static types and C functions to remove "yellow" lines.

#### If you use a variable only as a C object, use primitive tipes 
cdef int namevar

def double namevar

cdef float namevar

#### If you call a function only within C code, use a specific declaration "cdef"

cdef function_name(self, int param1, double param2):
...



## Step6: Iterate step 4 and 5 until you are satisfied with how clean your code is, then compile. An example of non optimized code can be found in the source folder of this notebook with the _SLOW suffix

## Step7: the compilation generates a file wose name is something like "SLIM_MSE_fastest.cp36-win_amd64.pyd" and tells you the source file, the architecture it is compiled for and the OS

## Step8: Import and use the compiled file as if it were any python object, function or class

In [ ]:
from Cython_examples.SLIM_MSE_fastest import train_multiple_epochs

train_multiple_epochs(URM_train, 1e-3, 1)

# A few warnings on ML algorithms

### - Why do we bother with KNNs if we have ML?
Because sometimes ML algorithms work better than heuristic ones, sometimes they do not

### - ML algorithms are always best because they learn from the data
Not really... Yes they learn from the data but the data is sometimes too noisy, too sparse and does not yeld good results. There is plenty of examples of cases where ML algorithms are not the best choice.

### - We should use this complex model because it can in theory approximate any function!!
Theory is important but... does it work in practice? Often complex modes are veeeery difficult to train and you need to use lots of tricks: adaptive gradients, data normalization, careful initialization and crafted batches...

### - I have trained my model for 2 epochs but the result is not great
Have you just used the default learning rate or have you optimized it? Why did you stop after 2 epochs? You may need 100s of epochs

### - If I select a high learning rate (maybe 1e-3) after 5 epochs I get a result wich is not very good, if I use a lower learning rate (maybe 1e-6) the result is much worse
Of course, the lower the learning rate the slower the training process but the best solution you may find. Again, you may need 100s of epochs

### - Training and optimizing the hyperparameters of this ML model takes several hours, what am I doing wrong?
Probably nothing, ML is computationally expensive and takes time... A few hours is a normal timespan. Sometimes the end result is still not satisfactory.